# Цели и задачи

**Цель** этого ipynb-файла и соответствующего Python отображения – генерировать html для соответствующей страницы articles.

# Текущие задачи

* Отдельно определять число просмотров теста
* Обрабатывать ошибки Connection Error -- делать пропуск и последующее обращение в конце списка до тех пор, пока список не будет целиком обновлён или ошибка не будет повторяться более 3 раз (статья удалена или что-то подобное)

In [1]:
import requests
headers = requests.utils.default_headers()
headers['User-Agent'] = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'

import pandas as pd
from bs4 import BeautifulSoup

Данные о статьях записаны в виде csv-файла. Файл содержит:
* `title` — заголовок статьи
* `url` — полная ссылка на публикацию
* `type` — тип публикации (перевод, статья, инструкция, курс или подборка)
* `date` — дата публикации
* `views` — число просмотров
* `main` — главный тег статьи
* `addtional` — дополнительные теги
* `raiting` — мой личный рейтинг отношения к статье
* `comment` — краткий комментарий

In [2]:
df = pd.read_csv("posts.csv")
df.tail()

,title,url,type,date,views,main,additional,rating,comment
105,Иллюстративный видеокурс по линейной алгебре: ...,https://proglib.io/p/algebra,подборка,2018-02-09,NaN,"Видео, курсы, ссылки",NaN,4,NaN
106,Как правильно искать и читать научные статьи?,https://proglib.io/p/research-papers,статья,2018-02-01,NaN,"Видео, курсы, ссылки",NaN,4,NaN
107,Создаем первый смарт-контракт на Ethereum мене...,https://proglib.io/p/smart-contract,перевод,2018-01-29,NaN,Блокчейн,NaN,3,NaN
108,Импульсные нейронные сети: мозг в компьютере,https://proglib.io/p/spiking-neural-networks,перевод,2018-01-22,NaN,Глубокое обучение,NaN,2,NaN
109,Как помочь ребенку выучить таблицу умножения,https://habr.com/ru/post/126043/,статья,2011-08-11,1000.0,Математика,NaN,5,NaN


# Просмотры

Данные о просмотрах обновляются, в особенности для новых статей, которые я писал для [Библиотеки программиста](https://proglib.io). Пропарсим страницы новых публикаций и обновим сведения о просмотрах.

In [3]:
df_proglib = df[df['url'].apply(lambda x: 'proglib' in x)]

In [4]:
def get_views_number(url:str):
    '''Возвращает число просмотров для переданного url'''
    page = requests.get(url, headers=headers).text
    soup = BeautifulSoup(page, 'html.parser')
    span = soup.body.find_all('span', {"class":"ml-1", "data-views":""})[0]
    
    return span.text

In [5]:
test_url = "https://proglib.io/tests/test-na-obshchie-znaniya-dlya-middle-python-razrabotchika"
df_proglib = df_proglib[df_proglib['url']!=test_url]

In [6]:
for url in df_proglib['url']:
    if pd.isnull(df.loc[df['url'] == url]['views'].values[0]):
        print(url)
        try:
            num = get_views_number(url)
            print(num)
            df.loc[df['url'] == url, ['views']] = num
        except:
            print("Error!")
            continue

https://proglib.io/p/kak-podruzhit-python-i-bazy-dannyh-sql-podrobnoe-rukovodstvo-2020-02-27
10627
https://proglib.io/p/python-i-dinamicheskoe-programmirovanie-na-primere-zadachi-o-ryukzake-2020-02-04
4135
https://proglib.io/p/kak-opublikovat-svoyu-python-biblioteku-na-pypi-2020-01-28
1734
https://proglib.io/p/veb-skraping-po-raspisaniyu-s-heroku-i-django-2020-01-17
5230
https://proglib.io/p/illyustrirovannoe-rukovodstvo-po-izmeneniyu-formy-mnogomernyh-massivov-numpy-2020-01-12
3482
https://proglib.io/p/iteriruemsya-pravilno-20-priemov-ispolzovaniya-v-python-modulya-itertools-2020-01-03
5987
https://proglib.io/p/ne-izobretat-velosiped-ili-obzor-modulya-collections-v-python-2019-12-15
5458
https://proglib.io/p/nazad-v-budushchee-prakticheskoe-rukovodstvo-po-puteshestviyu-vo-vremeni-s-python-2019-12-01
3533
https://proglib.io/p/10-sovetov-o-tom-kak-nauchit-rebenka-programmirovat-na-python-2019-10-12
9928
https://proglib.io/p/pishem-programmu-dlya-avtomaticheskogo-raspoznavaniya-obektov-s

In [7]:
df['views'].astype(float).sum()

3436475.0